<a href="https://colab.research.google.com/github/ansar2019/image-captionig/blob/main/Florence_2_DOCCI_FT_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate

In [2]:
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import requests


In [3]:
def caption(image):
  base_model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True)
  processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True)
  prompt = "<MORE_DETAILED_CAPTION>"
  adapter_name = "NikshepShetty/Florence-2-DOCCI-FT"
  model = PeftModel.from_pretrained(base_model, adapter_name, trust_remote_code=True)
  inputs = processor(text=prompt, images=image, return_tensors="pt")

  generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=1024,
      do_sample=False,
      num_beams=3
  )
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

  parsed_answer = processor.post_process_generation(generated_text, task="<MORE_DETAILED_CAPTION>", image_size=(image.width, image.height))

  print(parsed_answer)



In [4]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)
caption(image)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

{'<MORE_DETAILED_CAPTION>': 'An outdoor view of a light green Volkswagen Beetle parked in front of a yellow building. There is a brown door on the right side of the beetle. There are two brown windows on the building behind the beetle, and there is a white trim around the door. The beetle is parked on a gray brick sidewalk.'}


In [5]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


In [6]:
!unzip "/content/tef.zip"

Archive:  /content/tef.zip
   creating: tef/
  inflating: tef/COCO_test2014_000000000016.jpg  
  inflating: tef/COCO_test2014_000000001371.jpg  
  inflating: tef/COCO_test2014_000000002340.jpg  
  inflating: tef/COCO_test2014_000000002439.jpg  
  inflating: tef/COCO_test2014_000000003032.jpg  
  inflating: tef/COCO_test2014_000000003362.jpg  
  inflating: tef/COCO_test2014_000000003901.jpg  
  inflating: tef/COCO_test2014_000000004053.jpg  
  inflating: tef/COCO_test2014_000000006499.jpg  
  inflating: tef/COCO_test2014_000000007295.jpg  
  inflating: tef/COCO_test2014_000000007334.jpg  


In [13]:
import glob
from PIL import Image
import torch

filenames = glob.glob("/content/tef/*.jpg")
filenames.sort()
for image in filenames:
    raw_image = Image.open(image)
    caption(raw_image)
    with open('florence-2.txt', 'a') as fp:
        fp.write(str([image])+str([parsed_answer]))
        fp.write('\n')

{'<MORE_DETAILED_CAPTION>': 'A close-up view of a baseball player swinging a baseball bat. He is wearing a white uniform with the number 12 on the front of it. He has a blue helmet on his head and a black belt around his waist. His cleats are black and white with brown and white laces. The baseball is in the air and he is swinging the bat towards the ball. A tall chain link fence can be seen behind the player. Part of a person wearing a yellow jacket is standing behind the fence.'}
{'<MORE_DETAILED_CAPTION>': "An outdoor view of an elephant standing in a grassy field. The elephant is facing to the left and has a long trunk with two white tusks sticking out of it's mouth. Its trunk is wrinkled and covered in brown dirt. The grass is short and green. There is a small bush to the right of the elephant. Behind the elephant is a large gray rock. There are many trees in the background."}
{'<MORE_DETAILED_CAPTION>': 'A close-up view of a baseball player swinging a baseball bat. He is wearing 

In [ ]:
import glob
from PIL import Image
import torch

filenames = glob.glob("/content/Drive/MyDrive/testdata/*.jpg")
filenames.sort()
for image in filenames:
    raw_image = Image.open(image)
    caption(raw_image)
    with open('Florence-2-DOCCI-FT.txt', 'a') as fp:
        fp.write(str([image])+str([parsed_answer]))
        fp.write('\n')

{'<MORE_DETAILED_CAPTION>': 'Two men are standing in front of a wooden podium with a microphone on top of it. The man on the left is wearing a pink tie and has his arms crossed. He is looking to the right. The other man is wearing glasses and a gray suit with a white tag on it. Behind the men is an American flag. A black chair is sitting on the floor behind the men. Part of a white wall can be seen in the background.'}
{'<MORE_DETAILED_CAPTION>': 'A group of children are sitting at a table. They are all looking at a white laptop that is sitting on the table in front of them. The children are wearing pink shirts and black shorts. There is a yellow table behind them with white laptops on top of it. There are books on a shelf behind the children.'}
{'<MORE_DETAILED_CAPTION>': 'Three women are sitting at a white table in a conference room. Two of the women are wearing eyeglasses, one is wearing a multicolored tank top, the other is wearing an olive green shirt, and the third woman is weari